In [15]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import random
import glob
import tqdm
import os
from PIL import Image

In [16]:
RANDOM_SEED = 0
DATA_PATH = "../../data/derived_data/items_with_floor/"
OUTPUT_PATH = "../../data/derived_data/data_augmented/"

if not os.path.exists(OUTPUT_PATH):
    os.makedirs(OUTPUT_PATH)

In [17]:
operations = [
    tf.keras.layers.RandomRotation(0.5, interpolation="nearest", seed=None),
    tf.keras.layers.RandomFlip(mode="horizontal_and_vertical", seed=None),
    tf.keras.layers.RandomHeight(0.5, interpolation='nearest', seed=None),
    tf.keras.layers.RandomWidth(0.5, interpolation='nearest', seed=None),
    # tf.keras.layers.RandomCrop(300, 250, seed=None),
    tf.keras.layers.RandomContrast(0.5, seed=None),
    tf.keras.layers.RandomBrightness(0.5, value_range=(0, 255), seed=None)
]

In [18]:
N_RANDOM_OPERATIONS_COMBINATIONS = 10
SAVE_IMAGE = True
IMG_SIZE = 128

resize_and_rescale = tf.keras.Sequential([
  tf.keras.layers.Resizing(IMG_SIZE, IMG_SIZE, interpolation="nearest"),
  tf.keras.layers.Rescaling(1./255)
])

def augment_image(img_path):
    img = Image.open(img_path)
    img_arr = np.array(img)

    # without extension
    img_filename = img_path.split("/")[-1][:-4]

    for i, op in enumerate(operations):
        img_arr_augmented = op(img_arr)

        if SAVE_IMAGE:
            os.makedirs(f"{OUTPUT_PATH}/{img_filename}", exist_ok=True)
            Image.fromarray(img_arr_augmented.numpy().astype(np.uint8), "RGB").save(f"{OUTPUT_PATH}/{img_filename}/{img_filename}_{str(i)}.png")

        # img_arr_augmented = resize_and_rescale(img_arr_augmented)
        # np.save(f"{OUTPUT_PATH}/{img_filename}_{str(i)}", img_arr_augmented.numpy().astype(np.float16))

    random_ops_combs = [random.sample(operations, k=random.randint(1,len(operations))) for _ in range(N_RANDOM_OPERATIONS_COMBINATIONS)]

    for i, random_ops in enumerate(random_ops_combs, start=i+1):
        img_arr_augmented = img_arr
        for op in random_ops:
            img_arr_augmented = op(img_arr_augmented)
        
        if SAVE_IMAGE:
            os.makedirs(f"{OUTPUT_PATH}/{img_filename}", exist_ok=True)
            Image.fromarray(img_arr_augmented.numpy().astype(np.uint8), "RGB").save(f"{OUTPUT_PATH}/{img_filename}/{img_filename}_{str(i)}.png")

        # img_arr_augmented = resize_and_rescale(img_arr_augmented)
        # np.save(f"{OUTPUT_PATH}/{img_filename}_{str(i)}", img_arr_augmented.numpy().astype(np.float16))

In [19]:
output = []
for img_path in tqdm.tqdm(glob.glob(DATA_PATH + "*")):
    augment_image(img_path)

100%|██████████| 716/716 [02:21<00:00,  5.05it/s]
